In [2]:
import numpy as np
import time
import networkx as nx
import matplotlib.pyplot as plt
import random
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister)

from qiskit_aer import QasmSimulator
from qiskit.circuit.library import MCMT
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
def increment_gate(circuit, qpos, qcoin):
    n = len(qpos)
    for i in range(n):
        circuit.mcx(qcoin[:]+qpos[i+1:], qpos[i], None, mode='noancilla')

def decrement_gate(circuit, qpos, qcoin):
    n = len(qpos)
    for i in range(n):
        if i+1 < n: circuit.x(qpos[i+1:])
        circuit.mcx(qcoin[:]+qpos[i+1:], qpos[i], None, mode='noancilla')
        if i+1 < n: circuit.x(qpos[i+1:])

def num_to_binary(num, n):
    binary_str = ''
    while num > 0:
        if num % 2 == 0:
            binary_str = '0' + binary_str
        else:
            binary_str = '1' + binary_str
        num //= 2  
    while len(binary_str) < n:
        binary_str = '0' + binary_str
   
    return binary_str

def step(circuit, qpos, qcoin, cpos):
    circuit.h(qcoin)
    circuit.barrier()
    # y operations
    increment_gate(circuit, qpos[len(qpos)//2:], qcoin)
    circuit.x(qcoin[0])
    decrement_gate(circuit, qpos[len(qpos)//2:], qcoin)
    # x operations
    circuit.x(qcoin)
    increment_gate(circuit, qpos[:len(qpos)//2], qcoin)
    circuit.x(qcoin[0])
    decrement_gate(circuit, qpos[:len(qpos)//2:], qcoin)
    circuit.barrier()
   
def origin_converter_2d_x_n(circuit, origin, n):
    str = num_to_binary(origin[0], n)
    for i in range(0, n):
        if str[i] == '0':
            continue
        circuit.x(i)
    return circuit

def origin_converter_2d_y_n(circuit, origin, n):
    str = num_to_binary(origin[1], n)
    for i in range(0, n):
        if str[i] == '0':
            continue
        circuit.x(n + i)
    return circuit

def binary_to_num(str, n):
    str = str[::-1]
    sum = 0
    for i in range(0, n):
        if str[i] == '1':
            sum = sum + 2**i
        else:
            continue
   
    return sum

def probabilities_calc(circuit, origin, last_position, steps, qpos, qcoin, cpos, n):
    num = random.random()
    backend = QasmSimulator()
    if(steps == 0):
        return last_position
    for i in range(steps):
        step(circuit, qpos, qcoin, cpos)
    # # Map the quantum measurement to the classical bits
    circuit.measure(qpos,cpos)
    counts = backend.run(circuit, shots=1024).result().get_counts()
    state = list(counts.keys())
    count_dict = {}

    exp = 2**n

    for i in range(0, exp):
        for j in range(0, exp):
            count_dict[str(i) + ":" + str(j)] = 0

    for i in state:
        count_dict[str(binary_to_num(i[n:][::-1], n)) + ":" + str(binary_to_num(i[:n][::-1], n))] = counts[i]

    i = last_position[0]
    j = last_position[1]

    s1 =  count_dict[str(step_last_position_original[(i-1)%exp][j][0]) + ":" + str(step_last_position_original[(i-1)%exp][j][1])]
    s2 =  count_dict[str(step_last_position_original[i][(j-1)%exp][0]) + ":" + str(step_last_position_original[i][(j-1)%exp][1])]
    s3 =  count_dict[str(step_last_position_original[(i+1)%exp][j][0]) + ":" + str(step_last_position_original[(i+1)%exp][j][1])]
    s4 =  count_dict[str(step_last_position_original[i][(j+1)%exp][0]) + ":" + str(step_last_position_original[i][(j+1)%exp][1])]

    sum = s1 + s2 + s3 + s4

    if(sum == 0):
        print("sum 0 reported")
        print("origin: " , origin)
        print("last_position: " , last_position)
        print("steps: ", steps)
        return [last_position[0] + 1, last_position[1] + 1]

    p1 = s1/sum
    p2 = s2/sum
    p3 = s3/sum
    p4 = s4/sum

    list_prob = [p1, p2, p3, p4]
    list_prob_cumilative = [p1, p1 + p2, p1 + p2 + p3, p1 + p2 + p3 + p4]

    if(0<= num < list_prob_cumilative[0]):
        return [(i-1)%exp, j]
    elif(list_prob_cumilative[0] <= num < list_prob_cumilative[1]):
        return [i, (j-1)%exp]
    elif(list_prob_cumilative[1] <= num < list_prob_cumilative[2]):
        return [(i+1)%exp, j]
    elif(list_prob_cumilative[2] <= num < list_prob_cumilative[3]):
        return [i, (j+1)%exp]

def run_2d_n(origin, steps, last_position, step_last_position_2d, n):
    qpos = QuantumRegister(2*n,'qc')
    qcoin = QuantumRegister(2,'qanc')
    cpos = ClassicalRegister(2*n,'cr')
    circuit = QuantumCircuit(qpos, qcoin, cpos)
    circuit = origin_converter_2d_x_n(circuit, origin, n)
    circuit = origin_converter_2d_y_n(circuit, origin, n)

    new_position = probabilities_calc(circuit, origin, last_position, steps, qpos, qcoin, cpos, n)
    step_last_position_2d[origin[0]][origin[1]] = new_position
    return new_position

def create_matrix_with_ones(L, N):
    matrix = [[0 for _ in range(L)] for _ in range(L)]
    all_positions = [(i, j) for i in range(L) for j in range(L)]
    selected_positions = random.sample(all_positions, N)
    for i, j in selected_positions:
        matrix[i][j] = 1
    return matrix

def check_if_all_zeros(check):
    for i in range(0, exp):
        for j in range(0, exp):
            if(check[i][j]==1):
                return False
    return True  

def plot_grid_improved(grid, is_agent, filename=None):
    """
    Optimized grid plotting function using imshow for large grids.
    """
    rows, cols = len(grid), len(grid[0])
   
    # Create RGB color array
    color_grid = np.zeros((rows, cols, 3))
   
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 0:
                color_grid[i, j] = [0, 0, 1] if is_agent[i][j] == 1 else [1, 1, 1]  # Blue or White
            elif grid[i][j] == 1:
                color_grid[i, j] = [1, 0, 0]  # Red
            elif grid[i][j] == 2:
                color_grid[i, j] = [0, 1, 0]  # Green
   
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(color_grid, interpolation='nearest')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title(f"Grid Size: {rows}x{cols}")
   
    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

def plot_visited_improved(grid, filename=None):
    rows, cols = len(grid), len(grid[0])
    color_grid = np.zeros((rows, cols, 3))
   
    for i in range(rows):
        for j in range(cols):
            color_grid[i, j] = [1, 1, 0] if grid[i][j] == 1 else [1, 1, 1]  # Yellow or White
   
    fig, ax = plt.subplots(figsize=(20, 20))
    ax.imshow(color_grid, interpolation='nearest')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title(f"Visited Cells - Grid Size: {rows}x{cols}")
   
    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

def walker2d_1(check, infection_count=0):
    iter = 0
    while not check_if_all_zeros(check):
        for i in range(exp):
            for j in range(exp):
                if check[i][j] == 1:
                    if step_counter[i][j] == tau_max:
                        step_counter[i][j] = 0
                        step_last_position_2d[i][j] = step_last_position_original[i][j]
                        check[i][j] = 2
                    else:
                        step_counter[i][j] += 1

        for i in range(exp):
            for j in range(exp):
                if check[i][j] == 1:
                    r_x_new, r_y_new = run_2d_n([i, j], step_counter[i][j], step_last_position_2d[i][j], step_last_position_2d, n)
                    is_visited[r_x_new][r_y_new] = 1
                    num = random.random()
                    if check[r_x_new][r_y_new] == 0 and num <= p and is_agent[r_x_new][r_y_new] == 1:
                        check[r_x_new][r_y_new] = 1
                        infection_count += 1  # Count new infection

        iter += 1
        print(iter)

    return infection_count

def calculate_r0(total_infections, num_initial_infected):
    """
    Calculates R₀ based on total infections and initial infected count
    """
    if num_initial_infected == 0:
        return 0
    return total_infections / num_initial_infected

# Main simulation loop
list_sum = []
list_r0 = []
for i in range(0, 10000):
    p = 0.9
    tau_max = 1
    n = 5
    exp = 2**n
    N = 1000
   
    check = [[0 for x in range(0, exp)] for y in range(0, exp)]
    is_agent = create_matrix_with_ones(exp, N)
    is_visited = [[0 for x in range(0, exp)] for y in range(0, exp)]
   
    # Set initial infection
    check[1][1] = 1
    is_agent[1][1] = 1
    num_initial_infected = 1  # Track number of initial infections
   
    step_last_position_2d = [[[0 for x in range(0, 2)] for y in range(0, exp)] for z in range(0, exp)]
    step_last_position_original = [[[0 for x in range(0, 2)] for y in range(0, exp)] for z in range(0, exp)]
    step_counter = [[0 for x in range(0, exp)] for y in range(0, exp)]

    for i in range(0, exp):
        for j in range(0, exp):
            step_last_position_2d[i][j][0] = i
            step_last_position_2d[i][j][1] = j

    for i in range(0, exp):
        for j in range(0, exp):
            step_last_position_original[i][j][0] = i
            step_last_position_original[i][j][1] = j

    # Run simulation and get total number of new infections
    total_infections = walker2d_1(check, 0)
   
    # Calculate R₀
    r0_value = calculate_r0(total_infections, num_initial_infected)
    list_r0.append(r0_value)
   
    cluster_size = 0
    for j in range(len(is_visited)):
        for k in range(len(is_visited[0])):
            if(is_visited[j][k] == 1):
                cluster_size += 1
   
    print("SIZE OF CLUSTER:", cluster_size)
    print(f"Total infections: {total_infections}")
    print(f"R₀ VALUE: {r0_value:.2f}")
    list_sum.append(cluster_size)

# Calculate average R₀
total_r0 = 0
for r0 in list_r0:
    total_r0 += r0
mean_r0 = total_r0 / len(list_r0)
std_r0 = np.std(list_r0)
print(f"\nAverage R₀: {mean_r0:.2f} ± {std_r0:.2f}")

# Plot R₀ distribution
plt.figure(figsize=(10, 6))
plt.hist(list_r0, bins=20, edgecolor='black')
plt.title(f'Distribution of R₀ Values\nMean = {mean_r0:.2f}, Std = {std_r0:.2f}')
plt.xlabel('R₀')
plt.ylabel('Frequency')
plt.plot()

1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
4
5
6
SIZE OF CLUSTER: 5
Total infections: 4
R₀ VALUE: 4.00


3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
7
SIZE OF CLUSTER: 6
Total infections: 5
R₀ VALUE: 5.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
5
6
7
SIZE OF CLUSTER: 6
Total infections: 5
R₀ VALUE: 5.00
1
2


1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
SIZE OF CLUSTER: 5
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
7
SIZE OF CLUSTER: 5
Total infections: 5
R₀ VALUE: 5.00
1
2
3
4
5
6
7
SIZE OF CLUSTER: 5
Total infections: 5
R₀ VALUE: 5.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VA

1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2


1
2
3
4
5
6
7
SIZE OF CLUSTER: 5
Total infections: 5
R₀ VALUE: 5.00
1
2
3
4
5
6
7
SIZE OF CLUSTER: 6
Total infections: 5
R₀ VALUE: 5.00
1
2
3
4
5
6
7
8
9
SIZE OF CLUSTER: 7
Total infections: 7
R₀ VALUE: 7.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE:

...

3
4
5
6
7
SIZE OF CLUSTER: 5
Total infections: 5
R₀ VALUE: 5.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
7
8
9
10
SIZE OF CLUSTER: 8
Total infections: 8
R₀ VALUE: 8.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
SIZE OF CLUSTER: 5
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 

2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
SIZE OF CLUSTER: 4
Total infections: 4
R₀ VALUE: 4.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
5
SIZE OF CLUSTER: 4
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
SIZE OF CLUSTER: 3
Total infections: 3
R₀ VALUE: 3.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
SIZE OF CLUSTE

2
3
4
SIZE OF CLUSTER: 3
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
4
5
6
7
8
SIZE OF CLUSTER: 6
Total infections: 6
R₀ VALUE: 6.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
4
5
6
7
8
9
SIZE OF CLUSTER: 7
Total infections: 7
R₀ VALUE: 7.00
1
2
3
4
SIZE OF CLUSTER: 2
Total infections: 2
R₀ VALUE: 2.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00
1
2
SIZE OF CLUSTER: 1
Total infections: 0
R₀ VALUE: 0.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.0

4
5
6
7
SIZE OF CLUSTER: 5
Total infections: 5
R₀ VALUE: 5.00
1
2
3
SIZE OF CLUSTER: 2
Total infections: 1
R₀ VALUE: 1.00


KeyboardInterrupt: 